# Using LangChain and GPT for AI-Powered Reviews Analysis

In this notebook, we'll demonstrate how to integrate LangChain with GPT for extracting and analyzing product reviews. This is a practical approach to extracting structured information from unstructured data, such as web pages.

## Setup and Initialization

First, let's install the necessary libraries. 


Then, let's import the necessary libraries and configure our environment.

In [ ]:
import asyncio
import nest_asyncio
from langchain.agents.agent_toolkits import PlayWrightBrowserToolkit
from langchain.tools.playwright.utils import create_async_playwright_browser
from langchain.chat_models import ChatOpenAI
from langchain.chains import create_extraction_chain
import os
from dotenv import load_dotenv, find_dotenv

# Load environment variables from the .env file.
load_dotenv(find_dotenv())

# Allow nested async calls.
nest_asyncio.apply()

## Web Scraping with PlayWright

We'll use the PlayWright toolkit to navigate to the desired Amazon product reviews URL and extract the reviews.

In [ ]:
async def fetch_reviews():
    async_browser = create_async_playwright_browser()
    toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)
    tools = toolkit.get_tools()

    tools_by_name = {tool.name: tool for tool in tools}
    navigate_tool = tools_by_name["navigate_browser"]
    get_elements_tool = tools_by_name["get_elements"]

    await navigate_tool.arun(
        {"url": "https://www.amazon.com/OnePlus-Unlocked-Android-Smartphone-Charging/product-reviews/B07XWGWPH5/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews"}
    )

    elements = await get_elements_tool.arun(
        {"selector": ".review", "attributes": ["innerText"]}
    )
    return elements

# Fetch reviews
reviews = asyncio.run(fetch_reviews())

## Data Extraction and Analysis with GPT

Now, we'll define the schema for the extracted reviews and use GPT to process this data.

In [ ]:
schema = {
    "properties": {
        "name": {"type": "string"},
        "review": {"type": "string"},
        "rating": {"type": "integer"},
    },
    "required": ["name", "review", "rating"],
}

llm = ChatOpenAI(
    temperature=0, 
    model="gpt-3.5-turbo",
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

chain = create_extraction_chain(schema, llm)
results = chain.run(reviews)

print(results)

## Conclusion

With the combined power of LangChain and GPT, we've efficiently extracted and processed product reviews from a webpage. This approach showcases the potential of integrating web scraping tools with advanced NLP models for various data analysis tasks.